In [1]:
#!python3 -m pip install biopython

In [2]:
from Bio import GenBank
from pathlib import Path
import pandas as pd

In [3]:
data_path = "/data/hiv/data"

In [4]:
def parse_record(record):
    obj = {}
    
    if 'HIV' in record.source:
        
#         if len(record.sequence) <= 200:
#             return None

        obj = {
            "accession": record.accession[0], 
            'length': len(record.sequence),
            'sequence': record.sequence       
        }

        #[Feature(key='source', location='1..1686'), Feature(key='gene', location='<1..>1686'), Feature(key='CDS', location='<1..>1686')]
        for feature in record.features:
            if feature.key == "source":
                # [Qualifier(key='/organism=', value='"Human immunodeficiency virus 1"'), Qualifier(key='/proviral', value=''), Qualifier(key='/mol_type=', value='"genomic DNA"'), Qualifier(key='/db_xref=', value='"taxon:11676"'), Qualifier(key='/country=', value='"Spain"')]
                for qualifier in feature.qualifiers:
                    if qualifier.key == "/country=":
                        obj["country"] = qualifier.value.replace('"','').replace("'", "")

            if feature.key == "gene":
                pass

            if feature.key == "CDS":
                gene = None
                sequence = None
                integrase = False
                
                for qualifier in feature.qualifiers:
                    if qualifier.key == "/gene=":
                        gene = qualifier.value.replace('"','').replace("'", "").lower()

                    if qualifier.key == "/translation=":
                        sequence = qualifier.value.replace('"','').replace("'", "")
                    
                    if qualifier.key == "/product=":
                        value = qualifier.value.replace('"','').replace("'", "") 
                        if value == "integrase":
                            integrase = True
                
                if sequence and integrase:
#                     print(f"{record.accession[0]} {sequence}")
                    obj[gene] = sequence    
    
    if len(obj.keys()) <= 4:
        # accession, length, sequence, country
        return None
    
    return obj 
    
    
def parse_file(filename):
    data = []
    with open(filename, "r") as handle:
        records = GenBank.parse(handle)
        while True:
            try:
                obj = parse_record(records.__next__())
                if obj:
                    data.append(obj)
            
            except StopIteration as e:
                break
                    
            except Exception as exc:
                print(exc)
                
    return data                

In [5]:
data = []
for filename in sorted(list(Path(data_path).glob("*.seq"))):
    print(f"Parsing {filename}")
    sub_data = parse_file(filename)
    if len(sub_data) > 0:
        data.extend(sub_data)
        sub_df = pd.DataFrame(sub_data)
        sub_df.to_csv(filename.with_suffix(".csv"))
        print("Done")
    else:
        print("No useful data")
        

df = pd.DataFrame(data)
df.to_csv(Path(data_path).joinpath("2-integrase.csv"), index = False)    

Parsing /data/hiv/data/gbvrl1.seq
Done
Parsing /data/hiv/data/gbvrl10.seq
Done
Parsing /data/hiv/data/gbvrl11.seq
No useful data
Parsing /data/hiv/data/gbvrl12.seq
Done
Parsing /data/hiv/data/gbvrl2.seq
Done
Parsing /data/hiv/data/gbvrl3.seq
Done
Parsing /data/hiv/data/gbvrl4.seq
No useful data
Parsing /data/hiv/data/gbvrl5.seq
No useful data
Parsing /data/hiv/data/gbvrl6.seq
No useful data
Parsing /data/hiv/data/gbvrl7.seq
No useful data
Parsing /data/hiv/data/gbvrl8.seq
Done
Parsing /data/hiv/data/gbvrl9.seq
Done


In [6]:
df

,accession,length,sequence,country,pol,NaN
0,AB716142,864,TTTTTAGATGGGATAGAKAAGGCTCAAGAAGAACATGAAAGATATC...,Iran:Tehran,FLDGIXKAQEEHERYHSNWRAMASDFNLPPIVAKEIVANCDKCQLK...,NaN
1,AB716143,864,TTCTTAGATGGAATAGATAAGGCTCAAGAAGAACATGAAAGATATC...,Iran:Tehran,FLDGIDKAQEEHERYHSNWRAMASDFNLPPVVAKQIVANCDKCQLK...,NaN
2,AB716144,864,TTTCTAGATGGAATAGATAAGGCTCAAGAAGACCATGAAAAATATC...,Iran:Tehran,FLDGIDKAQEDHEKYHSNWRAMASDFNLPPIVAKEIVANCDKCQLK...,NaN
3,AB716145,864,TTTTTAGATGGAATAGATAAGGCTCAAGAAGAACATGAAAGATATC...,Iran:Tehran,FLDGIDKAQEEHERYHSNWRAMASDFNLPPVIAKEIVASCNKCQJK...,NaN
4,AB716146,864,TTTTTAGATGGAATAGATAAGGCTCAAGAAGAACATGAAAAATATC...,Iran:Tehran,FLDGIDKAQEEHEKYHNNWRAMASDFNLPPVVAKQIVANCDKCQLK...,NaN
...,...,...,...,...,...,...
5093,EU517892,867,TTTTTAGATGGGATAGATAAGGCCCAAGAAGAACATGARAAATATC...,USA,FLDGIDKAQEEHEKYHSNWRAMASDFNLPPVVAKEIVASCDKCQLK...,NaN
5094,EU517894,867,TTTTTAGATGGAATAGATAAGGCCCAAGAGGAACATGAAAAGTATC...,USA,FLDGIDKAQEEHEKYHSNWRAMASDFNLPPVVAKEIVASCDKCQLK...,NaN
5095,EU517895,867,TTTTTAGATGGAATAGATAAGGCCCAAGAAGAACATGAAAAATATC...,USA,FLDGIDKAQEEHEKYHSNWRAMASDFNLPPVVAKEIVASCDKCQLK...,NaN
5096,EU517896,867,TTTTTAGATGGAATAGATAAAGCCCAAGAAGAACATGARAAATATC...,USA,FLDGIDKAQEEHEKYHSNWRAMASDFNLPPVVAKEIVASCDKCQLK...,NaN
